In [27]:
import boto3
import pandas as pd
from IPython.display import display # 用于在 Notebook 中显示表格

# --- 1. 配置参数 ---
# ⚠️ 替换为您要查询的实例ID，以及您的 AWS 区域
INSTANCE_ID = "i-0901772c77b7aa090" 
REGION = "us-east-1"
# --------------------

ec2_client = boto3.client('ec2', region_name=REGION)
SECURITY_GROUP_ID = None

# --- 2. 实例基本信息 ---
print("--- 实例基本信息 ---")

try:
    # 获取实例信息
    response = ec2_client.describe_instances(InstanceIds=[INSTANCE_ID])
    instance_info = response['Reservations'][0]['Instances'][0]

    # 提取基本信息
    instance_data = {
        'ID': [instance_info.get('InstanceId')],
        'State': [instance_info.get('State', {}).get('Name')],
        'IP': [instance_info.get('PublicIpAddress')],
        # 提取 Name Tag
        'Name': [next((tag['Value'] for tag in instance_info.get('Tags', []) if tag['Key'] == 'Name'), None)]
    }

    # 显示基本信息表格
    display(pd.DataFrame(instance_data))

    # 提取安全组ID (用于下一步)
    security_groups = instance_info.get('SecurityGroups', [])
    if security_groups:
        SECURITY_GROUP_ID = security_groups[0]['GroupId']
    
except Exception as e:
    print(f"获取实例信息失败，请检查 ID 和权限: {e}")
    # 如果实例不存在或权限不足，则跳过后续步骤
    SECURITY_GROUP_ID = None 


# --- 3. 查询安全组的入站规则 (端口信息) ---
if SECURITY_GROUP_ID:
    print("\n--- 实例开放的入站端口 (安全组 ID: {}) ---".format(SECURITY_GROUP_ID))
    
    try:
        # 查询安全组详情
        sg_response = ec2_client.describe_security_groups(GroupIds=[SECURITY_GROUP_ID])
        
        # 提取 IpPermissions
        ip_permissions = sg_response['SecurityGroups'][0]['IpPermissions']
        
        port_rules = []
        for rule in ip_permissions:
            protocol = rule.get('IpProtocol', 'N/A')
            
            # 提取源 IP/SG 信息
            sources = []
            if rule.get('IpRanges'):
                sources.extend(r.get('CidrIp') for r in rule['IpRanges'])
            if rule.get('UserIdGroupPairs'):
                sources.extend(f"SG ID: {g.get('GroupId')}" for g in rule['UserIdGroupPairs'])

            # 收集规则数据
            port_rules.append({
                'Protocol': protocol,
                'FromPort': rule.get('FromPort', 'ALL'),
                'ToPort': rule.get('ToPort', 'ALL'),
                'Source': ', '.join([s for s in sources if s is not None]) or 'N/A'
            })

        if port_rules:
            display(pd.DataFrame(port_rules))
        else:
            print("该安全组没有开放的入站规则。")

    except Exception as e:
        print(f"获取安全组信息失败: {e}")
else:
    print("未找到实例的安全组信息，跳过端口查询。")

/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


--- 实例基本信息 ---


,ID,State,IP,Name
0,i-0901772c77b7aa090,running,98.92.250.239,sleeping_disorder



--- 实例开放的入站端口 (安全组 ID: sg-0641448e5a6106e98) ---


,Protocol,FromPort,ToPort,Source
0,-1,ALL,ALL,SG ID: sg-0641448e5a6106e98


In [9]:
import boto3
import os
import pandas as pd
from botocore.exceptions import ClientError
from IPython.display import display

# ================= 配置区域 (根据您的实际情况修改) =================
# 您的 AWS 区域
AWS_REGION = "us-east-1"
# 您的 ECR 仓库名称
ECR_REPO_NAME = "sleep-predictor"
# 您的 EC2 实例 ID
EC2_INSTANCE_ID = "i-06cc58e920e71518a"
# 本地 SSH 密钥路径 (用于验证文件是否存在)
LOCAL_SSH_KEY_PATH = "key.pem"  # <--- 请修改为您的密钥实际路径，例如 "/Users/xxx/key.pem"
# ===============================================================

# 初始化客户端
ecr_client = boto3.client('ecr', region_name=AWS_REGION)
ec2_client = boto3.client('ec2', region_name=AWS_REGION)

print(f"🔍 开始检查资源状态 (Region: {AWS_REGION})...\n")

# -----------------------------------------------------------
# 1. ECR 仓库信息检查
# -----------------------------------------------------------
print("--- [1] ECR 仓库信息 ---")
ecr_uri = None
try:
    response = ecr_client.describe_repositories(repositoryNames=[ECR_REPO_NAME])
    repo_info = response['repositories'][0]
    ecr_uri = repo_info['repositoryUri']
    
    print(f"✅ 仓库状态: 存在")
    print(f"📦 仓库名称: {ECR_REPO_NAME}")
    print(f"🔗 仓库 URI: {ecr_uri}")
    print(f"🆔 Registry ID: {repo_info['registryId']}")
except ClientError as e:
    print(f"❌ ECR 检查失败: {e}")

print("-" * 30)

# -----------------------------------------------------------
# 2. 镜像推送及登录信息 (模拟检查)
# -----------------------------------------------------------
print("--- [2] 镜像与登录权限检查 ---")
try:
    # 检查是否能获取登录 Token (这证明 IAM 权限没问题)
    auth_resp = ecr_client.get_authorization_token()
    print("✅ ECR 登录权限: 正常 (已获取 Auth Token)")
    
    # 检查现有镜像
    img_resp = ecr_client.list_images(repositoryName=ECR_REPO_NAME, maxResults=5)
    image_ids = img_resp.get('imageIds', [])
    
    if image_ids:
        print(f"✅ 现有镜像数量: {len(image_ids)}")
        # 整理成表格显示最近的 Tags
        img_data = []
        for img in image_ids:
            img_data.append({
                'ImageTag': img.get('imageTag', '<无Tag>'),
                'Digest': img.get('imageDigest')[:12] + "..."
            })
        display(pd.DataFrame(img_data))
    else:
        print("⚠️ 仓库是空的 (无镜像)")
        
except ClientError as e:
    print(f"❌ 权限或镜像检查失败: {e}")

print("-" * 30)

# -----------------------------------------------------------
# 3. EC2 实例信息
# -----------------------------------------------------------
print("--- [3] EC2 实例状态 ---")
public_ip = None
try:
    ec2_resp = ec2_client.describe_instances(InstanceIds=[EC2_INSTANCE_ID])
    instance = ec2_resp['Reservations'][0]['Instances'][0]
    
    public_ip = instance.get('PublicIpAddress', '未分配')
    state = instance['State']['Name']
    
    print(f"💻 实例名称: {next((tag['Value'] for tag in instance.get('Tags', []) if tag['Key'] == 'Name'), 'N/A')}")
    print(f"🆔 实例 ID: {EC2_INSTANCE_ID}")
    print(f"⚡️ 运行状态: {state}")
    print(f"🌐 公网 IP: {public_ip}")
    print(f"🏗️ 架构平台: {instance['Architecture']} (这将决定 Docker 构建时的 --platform 参数)")

except ClientError as e:
    print(f"❌ EC2 检查失败: {e}")

print("-" * 30)

# -----------------------------------------------------------
# 4. CD / SSH 配置信息汇总
# -----------------------------------------------------------
print("--- [4] CI/CD 变量配置清单 (GitHub Secrets) ---")
print("请根据以下状态去 GitHub 配置 Secrets:\n")

# 变量 1: EC2_HOST
if public_ip and public_ip != '未分配':
    print(f"✅ [EC2_HOST]     状态: 就绪 (Value: {public_ip})")
else:
    print(f"❌ [EC2_HOST]     状态: 缺失 (请确保实例已启动并有公网IP)")

# 变量 2: EC2_USER
print(f"✅ [EC2_USER]     状态: 默认 (Value: ubuntu) -> 假定使用 Ubuntu AMI")

# 变量 3: EC2_SSH_KEY
if os.path.exists(LOCAL_SSH_KEY_PATH):
    print(f"✅ [EC2_SSH_KEY]  状态: 本地已找到密钥文件 ({LOCAL_SSH_KEY_PATH})")
    print("                   👉 请将该文件内容全部复制到 GitHub Secret")
else:
    print(f"❌ [EC2_SSH_KEY]  状态: 本地未找到文件 ({LOCAL_SSH_KEY_PATH})")
    print("                   👉 请确认路径是否正确")

# 变量 4: AWS Credentials
print(f"ℹ️ [AWS_AK/SK]    状态: 假定已配置 (CI 构建镜像需要)")

print("\n🎉 检查结束。如果以上所有项均为 ✅，您可以开始编写 CI/CD 流程。")

/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


🔍 开始检查资源状态 (Region: us-east-1)...

--- [1] ECR 仓库信息 ---
✅ 仓库状态: 存在
📦 仓库名称: sleep-predictor
🔗 仓库 URI: 137568342316.dkr.ecr.us-east-1.amazonaws.com/sleep-predictor
🆔 Registry ID: 137568342316
------------------------------
--- [2] 镜像与登录权限检查 ---
✅ ECR 登录权限: 正常 (已获取 Auth Token)
✅ 现有镜像数量: 5


,ImageTag,Digest
0,<无Tag>,sha256:3f00d...
1,<无Tag>,sha256:5e118...
2,<无Tag>,sha256:f51dc...
3,<无Tag>,sha256:b2686...
4,<无Tag>,sha256:47a43...


------------------------------
--- [3] EC2 实例状态 ---
💻 实例名称: sleeping_disorder
🆔 实例 ID: i-06cc58e920e71518a
⚡️ 运行状态: running
🌐 公网 IP: 52.90.18.194
🏗️ 架构平台: x86_64 (这将决定 Docker 构建时的 --platform 参数)
------------------------------
--- [4] CI/CD 变量配置清单 (GitHub Secrets) ---
请根据以下状态去 GitHub 配置 Secrets:

✅ [EC2_HOST]     状态: 就绪 (Value: 52.90.18.194)
✅ [EC2_USER]     状态: 默认 (Value: ubuntu) -> 假定使用 Ubuntu AMI
❌ [EC2_SSH_KEY]  状态: 本地未找到文件 (key.pem)
                   👉 请确认路径是否正确
ℹ️ [AWS_AK/SK]    状态: 假定已配置 (CI 构建镜像需要)

🎉 检查结束。如果以上所有项均为 ✅，您可以开始编写 CI/CD 流程。


In [ ]:
# 直接设置 IP 和 用户名
!gh secret set EC2_HOST -b "52.90.18.194"
!gh secret set EC2_USER -b "ubuntu"

In [11]:
import boto3
import os
import time
import pandas as pd
from IPython.display import display
from botocore.exceptions import ClientError
import subprocess

# ================= 配置区域 (根据您的实际情况修改) =================
AWS_REGION = "us-east-1"
EC2_INSTANCE_ID = "i-06cc58e920e71518a"  # 替换为你的 EC2 实例 ID
NEW_KEY_PAIR_NAME = "my-new-ec2-key"    # 你希望新密钥对的名称
NEW_KEY_FILE_PATH = "my-new-ec2-key.pem" # 新密钥文件将保存到这里
# ===============================================================

ec2_client = boto3.client('ec2', region_name=AWS_REGION)

# 辅助函数，用于获取实例状态并显示为表格
def get_instance_status(instance_id):
    """获取指定 EC2 实例的状态、IP、名称和密钥对名称，并返回 DataFrame, 状态字符串, IP字符串, 密钥对名称字符串。"""
    try:
        response = ec2_client.describe_instances(InstanceIds=[instance_id])
        instance_info = response['Reservations'][0]['Instances'][0]
        instance_data = {
            'ID': [instance_info.get('InstanceId')],
            'State': [instance_info.get('State', {}).get('Name')],
            'IP': [instance_info.get('PublicIpAddress', 'N/A')], # 如果没有公网IP，显示N/A
            'Name': [next((tag['Value'] for tag in instance_info.get('Tags', []) if tag['Key'] == 'Name'), None)],
            'KeyPair': [instance_info.get('KeyName', 'N/A')] # 添加密钥对信息
        }
        df = pd.DataFrame(instance_data)
        state_name = instance_info.get('State', {}).get('Name')
        public_ip = instance_info.get('PublicIpAddress')
        key_name = instance_info.get('KeyName') # 获取密钥对名称
        return df, state_name, public_ip, key_name # 返回密钥对名称
    except ClientError as e:
        print(f"❌ 获取实例信息失败 ({instance_id})，请检查 ID 和权限: {e}")
        return pd.DataFrame(), 'error', None, None
    except IndexError:
        print(f"❌ 实例 ID '{instance_id}' 未找到。")
        return pd.DataFrame(), 'not_found', None, None

# --- Cell 1: 创建新的 EC2 密钥对 ---
print(f"--- 1. 处理 EC2 密钥对: {NEW_KEY_PAIR_NAME} ---")

if os.path.exists(NEW_KEY_FILE_PATH):
    print(f"ℹ️ 本地密钥文件 '{NEW_KEY_FILE_PATH}' 已存在。将使用此密钥对。")
    os.chmod(NEW_KEY_FILE_PATH, 0o400) # 确保权限是 400
    try:
        # 验证密钥对是否也在 AWS 中存在
        key_pairs_resp = ec2_client.describe_key_pairs(KeyNames=[NEW_KEY_PAIR_NAME])
        print(f"✅ 密钥对 '{NEW_KEY_PAIR_NAME}' 也存在于 AWS (KeyPairId: {key_pairs_resp['KeyPairs'][0]['KeyPairId']})。")
    except ClientError as e:
        if "InvalidKeyPair.NotFound" in str(e):
            print(f"⚠️ 警告：本地密钥文件 '{NEW_KEY_FILE_PATH}' 存在，但同名密钥对 '{NEW_KEY_PAIR_NAME}' 未在 AWS 中找到。请确认该密钥对的公钥是否已手动导入到 AWS。")
        else:
            print(f"❌ 检查 AWS 中密钥对失败: {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")
else:
    # 如果本地文件不存在，尝试在 AWS 中创建
    print(f"本地密钥文件 '{NEW_KEY_FILE_PATH}' 不存在。尝试在 AWS 中创建新的密钥对。")
    try:
        response = ec2_client.create_key_pair(KeyName=NEW_KEY_PAIR_NAME)
        key_material = response['KeyMaterial']

        with open(NEW_KEY_FILE_PATH, 'w') as f:
            f.write(key_material)

        os.chmod(NEW_KEY_FILE_PATH, 0o400)
        print(f"✅ 密钥对 '{NEW_KEY_PAIR_NAME}' 已在 AWS 中创建。私钥已保存到 '{NEW_KEY_FILE_PATH}' 并设置了正确权限 (chmod 400)。")

        # 验证密钥对是否已创建
        key_pairs_resp = ec2_client.describe_key_pairs(KeyNames=[NEW_KEY_PAIR_NAME])
        print(f"验证：新密钥对存在于 AWS (KeyPairId: {key_pairs_resp['KeyPairs'][0]['KeyPairId']})。")

    except ClientError as e:
        if "already exists" in str(e):
            print(f"⚠️ 密钥对 '{NEW_KEY_PAIR_NAME}' 已存在于 AWS，但本地文件缺失。请确保你有该文件的私钥。如果没有，请删除 AWS 中的密钥对，或更改 `NEW_KEY_PAIR_NAME` 重新创建。")
        else:
            print(f"❌ 创建密钥对失败: {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")


# --- Cell 2: 获取 EC2 实例当前状态 ---
print("\n--- 2. 获取 EC2 实例当前状态 ---")
# 修改函数调用以接收新的返回值
current_status_df, initial_state, initial_ip, _ = get_instance_status(EC2_INSTANCE_ID)
if not current_status_df.empty and initial_state != 'error' and initial_state != 'not_found':
    display(current_status_df)
    print(f"✅ 实例 '{EC2_INSTANCE_ID}' 当前状态: {initial_state}")
else:
    print(f"❌ 无法获取实例 '{EC2_INSTANCE_ID}' 的初始状态。请检查ID和权限。")
    initial_state = 'error' # 设置一个标志，阻止后续操作


# --- Cell 3: 停止 EC2 实例 ---
if initial_state == 'running':
    print(f"\n--- 3. 停止 EC2 实例: {EC2_INSTANCE_ID} ---")
    try:
        ec2_client.stop_instances(InstanceIds=[EC2_INSTANCE_ID])
        print("💡 停止实例命令已发送。正在等待实例变为 'stopped' 状态...")

        # 使用 boto3 的 waiter 轮询检查实例状态
        waiter = ec2_client.get_waiter('instance_stopped')
        waiter.wait(InstanceIds=[EC2_INSTANCE_ID])
        print(f"✅ 实例 '{EC2_INSTANCE_ID}' 已成功停止。")
    except ClientError as e:
        print(f"❌ 停止实例失败: {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")
elif initial_state == 'stopped':
    print(f"\n--- 3. 实例 '{EC2_INSTANCE_ID}' 当前状态已是 'stopped'，无需停止。跳过停止步骤。---")
else:
    print(f"\n--- 3. 实例 '{EC2_INSTANCE_ID}' 状态为 '{initial_state}'，无法停止。跳过停止步骤。---")


# --- Cell 4: 替换密钥对 ---
print(f"\n--- 4. 替换 EC2 实例 '{EC2_INSTANCE_ID}' 的密钥对为 '{NEW_KEY_PAIR_NAME}' ---")
# 替换密钥对前，再次确认实例是否已停止，并获取当前密钥对信息
_, current_state_for_replace, _, current_key_pair = get_instance_status(EC2_INSTANCE_ID)

if current_state_for_replace == 'stopped':
    if current_key_pair and current_key_pair != 'N/A':
        if current_key_pair == NEW_KEY_PAIR_NAME:
            print(f"ℹ️ 实例 '{EC2_INSTANCE_ID}' 已绑定相同的密钥对 '{NEW_KEY_PAIR_NAME}'，无需重复替换。")
            # 如果已经绑定了相同的密钥，我们可以选择跳过 modify_instance_attribute 调用。
            # 但为了幂等性，调用一次通常也无妨。这里让它继续执行 try/except 块。
        else:
            print(f"ℹ️ 实例 '{EC2_INSTANCE_ID}' 当前绑定密钥对为 '{current_key_pair}'，将替换为 '{NEW_KEY_PAIR_NAME}'。")
    else:
        print(f"ℹ️ 实例 '{EC2_INSTANCE_ID}' 当前未绑定密钥对，将直接绑定 '{NEW_KEY_PAIR_NAME}'。")

    try:
        # 使用 modify_instance_attribute 来绑定或替换密钥对
        response = ec2_client.modify_instance_attribute(
            InstanceId=EC2_INSTANCE_ID,
            Attribute='keyPair',
            Value=NEW_KEY_PAIR_NAME
        )
        print(f"✅ 实例 '{EC2_INSTANCE_ID}' 的密钥对已成功绑定/替换为 '{NEW_KEY_PAIR_NAME}'。")
    except ClientError as e:
        print(f"❌ 绑定/替换密钥对失败: {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")
else:
    print(f"❌ 实例 '{EC2_INSTANCE_ID}' 尚未停止 (当前状态: {current_state_for_replace})。请等待其停止后再执行此步骤。跳过密钥对操作。")


# --- Cell 5: 启动 EC2 实例 ---
print(f"\n--- 5. 启动 EC2 实例: {EC2_INSTANCE_ID} ---")
# 启动前，再次确认实例是否已停止或已替换密钥（如果上一步成功）
# 修改函数调用以接收新的返回值
_, current_state_for_start, _, _ = get_instance_status(EC2_INSTANCE_ID)
if current_state_for_start == 'stopped':
    try:
        ec2_client.start_instances(InstanceIds=[EC2_INSTANCE_ID])
        print("💡 启动实例命令已发送。正在等待实例变为 'running' 状态...")

        # 使用 boto3 的 waiter 轮询检查实例状态
        waiter = ec2_client.get_waiter('instance_running')
        waiter.wait(InstanceIds=[EC2_INSTANCE_ID])
        print(f"✅ 实例 '{EC2_INSTANCE_ID}' 已成功启动。")

        # 获取启动后的新状态和 IP 地址
        # 修改函数调用以接收新的返回值
        final_status_df, final_state, new_public_ip, _ = get_instance_status(EC2_INSTANCE_ID)
        if not final_status_df.empty:
            display(final_status_df)
            print(f"✅ 实例 '{EC2_INSTANCE_ID}' 最终状态: {final_state}, 新公网 IP: {new_public_ip}")
        else:
            print(f"❌ 无法获取实例 '{EC2_INSTANCE_ID}' 的最终状态。")

    except ClientError as e:
        print(f"❌ 启动实例失败: {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")
else:
    print(f"❌ 实例 '{EC2_INSTANCE_ID}' 状态为 '{current_state_for_start}'，无法启动。跳过启动步骤。")
    new_public_ip = None # 确保如果未启动则 IP 为 None


# --- Cell 6: 设置 GitHub Secrets ---
print("\n--- 6. 设置 GitHub Secrets ---")

# 设置 EC2_HOST
if 'new_public_ip' in locals() and new_public_ip and new_public_ip != 'N/A':
    print(f"设置 EC2_HOST 为: {new_public_ip}")
    try:
        result = subprocess.run(['gh', 'secret', 'set', 'EC2_HOST', '-b', new_public_ip], check=True, capture_output=True, text=True)
        print("✅ EC2_HOST secret 已设置。")
        # print(f"gh output: {result.stdout.strip()}") # 如果需要查看 gh 的标准输出
    except subprocess.CalledProcessError as e:
        print(f"❌ 设置 EC2_HOST secret 失败: {e.stderr.strip()}")
    except FileNotFoundError:
        print("❌ 'gh' 命令未找到。请确保 GitHub CLI 已安装并配置在 PATH 中。")
else:
    print("❌ 无法获取到 EC2 实例的公网 IP，跳过 EC2_HOST secret 设置。")

# 设置 EC2_USER
print("设置 EC2_USER 为: ubuntu")
try:
    result = subprocess.run(['gh', 'secret', 'set', 'EC2_USER', '-b', 'ubuntu'], check=True, capture_output=True, text=True)
    print("✅ EC2_USER secret 已设置。")
except subprocess.CalledProcessError as e:
    print(f"❌ 设置 EC2_USER secret 失败: {e.stderr.strip()}")
except FileNotFoundError:
    print("❌ 'gh' 命令未找到。请确保 GitHub CLI 已安装并配置在 PATH 中。")

# 设置 EC2_SSH_KEY
if os.path.exists(NEW_KEY_FILE_PATH):
    print(f"设置 EC2_SSH_KEY (从文件 '{NEW_KEY_FILE_PATH}')")
    try:
        with open(NEW_KEY_FILE_PATH, 'r') as f:
            ssh_key_content = f.read()
        
        # 使用 subprocess.run 传递文件内容
        result = subprocess.run(['gh', 'secret', 'set', 'EC2_SSH_KEY', '-b', ssh_key_content], check=True, capture_output=True, text=True)
        print("✅ EC2_SSH_KEY secret 已设置。")
    except subprocess.CalledProcessError as e:
        print(f"❌ 设置 EC2_SSH_KEY secret 失败: {e.stderr.strip()}")
    except FileNotFoundError:
        print("❌ 'gh' 命令未找到。请确保 GitHub CLI 已安装并配置在 PATH 中。")
    except Exception as e:
        print(f"❌ 读取或设置 EC2_SSH_KEY 失败: {e}")
else:
    print(f"❌ 本地未找到密钥文件 '{NEW_KEY_FILE_PATH}'，跳过 EC2_SSH_KEY secret 设置。请确保文件存在。")

print("\n🎉 所有 GitHub Secrets 设置步骤完成。")

/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


--- 1. 处理 EC2 密钥对: my-new-ec2-key ---
ℹ️ 本地密钥文件 'my-new-ec2-key.pem' 已存在。将使用此密钥对。
✅ 密钥对 'my-new-ec2-key' 也存在于 AWS (KeyPairId: key-019da3e1000f3e59b)。

--- 2. 获取 EC2 实例当前状态 ---


,ID,State,IP,Name,KeyPair
0,i-06cc58e920e71518a,running,18.208.214.192,sleeping_disorder,N/A


✅ 实例 'i-06cc58e920e71518a' 当前状态: running

--- 3. 停止 EC2 实例: i-06cc58e920e71518a ---
💡 停止实例命令已发送。正在等待实例变为 'stopped' 状态...
✅ 实例 'i-06cc58e920e71518a' 已成功停止。

--- 4. 替换 EC2 实例 'i-06cc58e920e71518a' 的密钥对为 'my-new-ec2-key' ---
ℹ️ 实例 'i-06cc58e920e71518a' 当前未绑定密钥对，将直接绑定 'my-new-ec2-key'。
❌ 绑定/替换密钥对失败: An error occurred (InvalidParameterValue) when calling the ModifyInstanceAttribute operation: Value (keyPair) for parameter attribute is invalid. Unknown attribute.

--- 5. 启动 EC2 实例: i-06cc58e920e71518a ---
💡 启动实例命令已发送。正在等待实例变为 'running' 状态...
✅ 实例 'i-06cc58e920e71518a' 已成功启动。


,ID,State,IP,Name,KeyPair
0,i-06cc58e920e71518a,running,54.83.106.11,sleeping_disorder,N/A


✅ 实例 'i-06cc58e920e71518a' 最终状态: running, 新公网 IP: 54.83.106.11

--- 6. 设置 GitHub Secrets ---
设置 EC2_HOST 为: 54.83.106.11
✅ EC2_HOST secret 已设置。
设置 EC2_USER 为: ubuntu
✅ EC2_USER secret 已设置。
设置 EC2_SSH_KEY (从文件 'my-new-ec2-key.pem')
✅ EC2_SSH_KEY secret 已设置。

🎉 所有 GitHub Secrets 设置步骤完成。


In [12]:
!gh secret list

]11;?\]11;?\NAME                   UPDATED               
AWS_ACCESS_KEY_ID      about 3 hours ago
AWS_REGION             about 3 hours ago
AWS_SECRET_ACCESS_KEY  about 3 hours ago
EC2_HOST               less than a minute ago
EC2_SSH_KEY            less than a minute ago
EC2_USER               less than a minute ago


STATUS  TITLE         WORKFLOW    BRANCH  EVENT  ID          ELAPSED  AGE       
X       use ec2 t...  Deploy ...  main    push   1995590...  1m3s     about 3...
X       test sage...  Deploy ...  main    push   1995263...  1m1s     about 3...
X       optimise ...  Deploy ...  main    push   1995235...  1m6s     about 3...
X       add CI pr...  Deploy ...  main    push   1995218...  12s      about 4...


In [ ]:
gh run list
gh run rerun 19957968444
gh run watch 19958407392
gh run view 19957968444 --log

ssh -i "project-final-key.pem" ubuntu@98.92.250.239
docker ps -a --filter name=sleep_app_container
docker logs sleep_app_container
sudo ss -tuln | grep 8000


REGION="us-east-1"
ACCOUNT_ID="137568342316"
ECR_REGISTRY="${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com"
ECR_REPOSITORY="sleep-predictor"
CONTAINER_NAME="sleep_app_container"
IMAGE_URI="${ECR_REGISTRY}/${ECR_REPOSITORY}:latest"
AWS_CLI_PATH="/home/ubuntu/.local/bin/aws"

$AWS_CLI_PATH ecr get-login-password --region $REGION | docker login --username AWS --password-stdin $ECR_REGISTRY

docker stop $CONTAINER_NAME || true
docker rm $CONTAINER_NAME || true

docker pull $IMAGE_URI

docker run -d \
  --restart always \
  -p 8000:8000 \
  --name $CONTAINER_NAME \
  $IMAGE_URI


1. 查看 Docker 服务状态
sudo systemctl status docker
docker ps
docker ps -a
docker logs <容器名称或ID>


curl http://98.92.250.239:8000/ping


]11;?\]11;?\Refreshing run status every 3 seconds. Press Ctrl+C to quit.

* main Deploy to EC2 · 19955903933
Triggered via push less than a minute ago

JOBS
* build-and-deploy (ID 57227046969)
  ✓ Set up job
  ✓ Build appleboy/ssh-action@v1.0.0
  ✓ Checkout code
  ✓ Configure AWS credentials
  ✓ Login to Amazon ECR
  * Build and Push Docker Image
  * Deploy to EC2 via SSH
  * Post Login to Amazon ECR
  * Post Configure AWS credentials
  * Post Checkout code
Refreshing run status every 3 seconds. Press Ctrl+C to quit.

* main Deploy to EC2 · 19955903933
Triggered via push less than a minute ago

JOBS
* build-and-deploy (ID 57227046969)
  ✓ Set up job
  ✓ Build appleboy/ssh-action@v1.0.0
  ✓ Checkout code
  ✓ Configure AWS credentials
  ✓ Login to Amazon ECR
  * Build and Push Docker Image
  * Deploy to EC2 via SSH
  * Post Login to Amazon ECR
  * Post Configure AWS credentials
  * Post Checkout code
Refreshing run status every 3 seconds. Press Ctrl+C to quit.

* main Deploy to EC2 ·

In [ ]:
这里放置信息，用来看目前ec2拉取镜像是哪里有问题
1.配置变量
2.拉取镜像
3.运行容器
4.访问测试

ubuntu@ip-172-31-73-115:~$ /home/ubuntu/.local/bin/aws ecr get-login-password --region $REGION | docker login --username AWS --password-stdin $ECR_REGISTRY

Unable to locate credentials. You can configure credentials by running "aws configure".

Error: Cannot perform an interactive login from a non TTY device

In [31]:
import requests
import json
import sys

# 部署在 EC2 上的模型 API 地址
# ---------------------------------------------
# ⚠️ 注意：请确保这是您最新 EC2 实例的 IP
EC2_IP = "98.92.250.239" 
PREDICT_URL = f"http://{EC2_IP}:8000/predict"
# ---------------------------------------------

# 1. 构造测试数据 (必须精确匹配您的 FastAPI 接口定义的请求体结构)
# 此示例数据基于常见的睡眠预测模型特征
test_data = {
  "Gender": "Male",
  "Age": 35,
  "Sleep_Duration": 6.8,
  "Quality_of_Sleep": 7,
  "Physical_Activity_Level": 45,
  "BMI_Category": "Normal",
  "Blood_Pressure": "120/80",
  "Heart_Rate": 70,
  "Daily_Steps": 6500
}

headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

print(f"尝试连接到: {PREDICT_URL}")
print("发送数据:", test_data)

try:
    # 2. 发送 POST 请求
    response = requests.post(PREDICT_URL, headers=headers, json=test_data, timeout=15)
    
    # 3. 检查响应状态
    # 如果状态码是 4xx 或 5xx，raise_for_status() 会抛出异常
    response.raise_for_status()

    # 4. 打印结果
    result = response.json()
    print("\n--- 预测结果 ---")
    print("状态码:", response.status_code)
    # 使用 json.dumps 格式化打印结果
    print("预测内容:", json.dumps(result, indent=4, ensure_ascii=False))

    if 'prediction' in result:
        print(f"\n模型预测结果: {result['prediction']}")

except requests.exceptions.RequestException as e:
    print("\n--- 错误 ---")
    print(f"请求失败或连接超时。请检查 EC2 是否运行。错误详情: {e}")
    # 如果有响应但状态码不是 200，也打印出来
    if 'response' in locals() and response.status_code != 200:
        print(f"HTTP 状态码: {response.status_code}")
        print(f"服务器响应: {response.text}")
    sys.exit(1)

尝试连接到: http://98.92.250.239:8000/predict
发送数据: {'Gender': 'Male', 'Age': 35, 'Sleep_Duration': 6.8, 'Quality_of_Sleep': 7, 'Physical_Activity_Level': 45, 'BMI_Category': 'Normal', 'Blood_Pressure': '120/80', 'Heart_Rate': 70, 'Daily_Steps': 6500}

--- 错误 ---
请求失败或连接超时。请检查 EC2 是否运行。错误详情: HTTPConnectionPool(host='98.92.250.239', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13d57ecd0>: Failed to establish a new connection: [Errno 61] Connection refused'))


AttributeError: 'dict' object has no attribute 'status_code'